In [2]:
from neo4j import GraphDatabase

In [3]:
from py2neo import Graph,Node,Relationship, NodeMatcher
from py2neo import cypher

In [4]:
import pandas as pd
import numpy as np


In [5]:
URI = "neo4j+s://67ec8314.databases.neo4j.io"
AUTH = ("neo4j", "ptdQYuJCwqQQt6BOO-qi323XZjT4G8wEnh90ohOrX3A")


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [7]:
graph = Graph(URI, auth=('neo4j', 'ptdQYuJCwqQQt6BOO-qi323XZjT4G8wEnh90ohOrX3A'))

In [8]:
test = pd.read_csv('data_test.csv', encoding='utf-8')
test.head(3)

,id события;ФИО участника события 1;ФИО участника события 2
0,189;Галчевская Карина Владимировна;Белоновская...
1,206;Офицеров Олег Романович;Сапожник Борис Вал...
2,445;Жандарова Лариса Германовна;Чемодуров Дами...


In [9]:
def parsing(df):
    """Function to split data 
    from one column into three
    columns
    """
    df[['event_id', 'event1_name', 'event2_name']] = pd.DataFrame([x.split(";") for x in df['id события;ФИО участника события 1;ФИО участника события 2'].tolist()])
    
    return df.iloc[:, 1:]

In [10]:
t = parsing(test)

In [11]:
t.head(3)

,event_id,event1_name,event2_name
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович


In [50]:
t.shape

(5000, 3)

In [12]:
t["index"] = t.index
t.head()

,event_id,event1_name,event2_name,index
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна,0
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич,1
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович,2
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович,3
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна,4


In [13]:
t['event1_name'].nunique()

4930

In [14]:
t['event2_name'].nunique()

4994

In [13]:
t["event_id"].nunique()


4985

In [14]:
trains=t["index"].unique()
len(trains)

5000

In [15]:
q1="""
MERGE(Z:Index{NAME:$index})
MERGE(A:ID{NAME:$event_id})
MERGE(B:NAME1{NAME:$event1_name})
MERGE(C:NAME2{NAME:$event2_name})
MERGE (A)-[:CONNECTED_TO]->(B)
MERGE (A)-[:CONNECTED_TO]->(C)
MERGE (B)-[:CONNECTED_TO]->(C)
MERGE (Z)-[:INDEX_OF]->(A)
MERGE (Z)-[:INDEX_OF]->(B)
MERGE (Z)-[:INDEX_OF]->(C)

"""

session = driver.session()
for tname in trains:
    x=t.groupby("index")[["index", "event_id"]].get_group(tname)["event_id"]
    y=t.groupby("index")[[ "index", "event1_name"]].get_group(tname)["event1_name"]
    z=t.groupby("index")[["index", "event2_name"]].get_group(tname)["event2_name"] 
    for i, j, k in zip(x, y, z):
        p={"index":tname, "event_id":i,"event1_name":j, "event2_name":k}
        
        session.run(q1,p)
    
    